In [ ]:
# Reference
#the code in this file are provided by the tutotial Lab2_Python.ipynb from https://github.com/akiratwang/MAST30034_Python
#
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import folium
from folium.plugins import FastMarkerCluster

# lat, long
COORDS = ['pickup_latitude', 'pickup_longitude']

# create an interactive geospatial graph
pickups_cluster = folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)
#pickups_cluster

In [ ]:
# read in the data
df = pd.read_feather("../preprocessed_data/taxi_add_weather_19.feather").drop('index', axis=1)

df.tail()

In [ ]:
df.groupby('PULocationID')['final_amount'].count().reset_index().sort_values('final_amount', ascending=False)

In [ ]:
df

In [ ]:
import geopandas as gpd

# sf stands for shape file
sf = gpd.read_file("../raw_data/taxi_zones/taxi_zones.shp")
zone = pd.read_csv("../raw_data/taxi_zones/taxi+_zone_lookup.csv")

# Convert the geometry shaape to to latitude and longitude
# Please attribute this if you are using it
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

In [ ]:
sf

In [ ]:
sf.head()

In [ ]:
gdf = gpd.GeoDataFrame(pd.merge(df, sf, left_on='PULocationID', right_on='LocationID')).drop('PULocationID',axis=1)
gdf.sample(2)

In [ ]:
geoJSON = gdf[['LocationID','geometry']].drop_duplicates('LocationID').to_json()

In [ ]:
m = folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)

# refer to the folium documentations on how to plot aggregated data.
m.add_child(folium.Choropleth(
    geo_data=geoJSON,
    name='choropleth',
))

m.save('../plots/foliumChoroplethMap.html')
m

In [ ]:
import json

# an example of what the geoJSON looks like
json.loads(geoJSON)

In [ ]:
gdf.loc[gdf['total_amount'] < 0]

In [ ]:
gdf[['LocationID','total_amount']].groupby('LocationID').sum().reset_index()

In [ ]:
m_trip_distance = folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)

# refer to the folium documentations on more information on how to plot aggregated data.
folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=gdf, # data source
    columns=['LocationID','total_amount'], # the columns required
    key_on='properties.LocationID', # this is from the geoJSON's properties
    fill_color='OrRd', # color scheme
    fill_opacity=0.9,
    line_opacity=0.5,
    legend_name='Trips' # legend title
).add_to(m_trip_distance)

m_trip_distance.save('../plots/foliumChoroplethMapTrips.html')
m_trip_distance

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_feather("../preprocessed_data/taxi_add_weather_19.feather").drop('index', axis=1)
# describe our data
df

In [ ]:
df[['trip_distance','fare_amount']].plot.scatter(x='trip_distance',
                                                  y='fare_amount')
plt.show()

In [ ]:
sns.distplot(df['fare_amount'], bins=30)

plt.show()

In [ ]:
from numpy import log, sqrt

# apply a log transformation for all x non-zero x points, else 0
def logify(x):
    return log(x) if x else 0

sns.distplot(df['fare_amount'].apply(logify), bins=50)
plt.show()

In [ ]:
sns.distplot(df['fare_amount'].apply(lambda x: log(x) if x else 0), bins=50)
plt.show()

In [ ]:
data = df.loc[df['trip_distance'] <= 15, 'trip_distance']

sns.distplot(data, bins=30)
plt.show()